In [19]:
import pandas as pd

In [20]:
def getHeatmapData(df, colname, step, outfile):
    min = df[colname].min()
    max = df[colname].max()
    quant = []
    for i in range(step):
        quant.append(i * (1 / step) * max)
    quant.append(max)
    df.groupby(['time', pd.cut(df[colname], quant)]).size().to_csv(outfile)

In [21]:
def getHeatmapDataAllStep(df, colname, outfile):
    min = df[colname].min()
    max = df[colname].max()
    quant = range(0, 33)
    
    df.groupby(['time', pd.cut(df[colname], quant)]).size().to_csv(outfile)

In [22]:
inputFile = './parsedData/trace_scsi.txt.json.calc.csv'
df = pd.read_csv(inputFile)
df['time'] = df['issue_time'].astype(int)
print(df.dtypes)

idx              int64
issue_time     float64
cmd_type        object
lba              int64
size(KB)         int64
latency(us)      int64
queue_cnt        int64
cpu_no           int64
host_no          int64
tag              int64
nextlba          int64
continuity       int64
time             int32
dtype: object


In [23]:
file_lba = './parsedData/heatmap_lba.csv'
file_queue = './parsedData/heatmap_queue.csv'
file_latency = './parsedData/heatmap_latency.csv'
file_latencypercentile = './parsedData/percentile_performance.csv'


# getHeatmapData(df, 'lba', 20, file_lba)
# getHeatmapData(df, 'latency(us)', 20, file_latency)
# getHeatmapDataAllStep(df, 'queue_cnt', file_queue)

df['lt99'] = df.groupby(['time'])['latency(us)'].transform(lambda x: x.quantile(q=0.99, interpolation='nearest'))
df['lt99_99'] = df.groupby(['time'])['latency(us)'].transform(lambda x: x.quantile(q=0.9999, interpolation='nearest'))
df['lt99_9999'] = df.groupby(['time'])['latency(us)'].transform(lambda x: x.quantile(q=0.999999, interpolation='nearest'))
df['lt100'] = df.groupby(['time'])['latency(us)'].transform(lambda x: x.quantile(q=1.0, interpolation='nearest'))
df['throughput'] = df.groupby(['time'])['size(KB)'].transform(lambda x: x.sum())
# getLatencyData(df, file_latency)
# 위 함수 수행 후, y축 값 엑셀에서 손으로 변경하고, tab -> comma 찾아 바꾸기 해야함


In [24]:
df.groupby(['time']).max().to_csv(file_latencypercentile)